In [ ]:
!python3 --version

Python 3.7.13


## Setup



GPU check

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May  2 14:54:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Install required packages

In [ ]:
!pip uninstall gym-jsbsim
!pip install stable-baselines3[extra]
!pip install tensorboardX
!pip install gym
!pip install matplotlib

Found existing installation: gym-jsbsim 0.6.7-4-g59abd7f
Can't uninstall 'gym-jsbsim'. No files were found to uninstall.


In [ ]:
!git clone https://github.com/asgv123/L2F_Challenge_YashGadhia.git

Cloning into 'L2F_Challenge_YashGadhia'...
remote: Enumerating objects: 6332, done.
remote: Counting objects: 100% (311/311), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 6332 (delta 146), reused 284 (delta 131), pack-reused 6021
Receiving objects: 100% (6332/6332), 22.39 MiB | 16.38 MiB/s, done.
Resolving deltas: 100% (3911/3911), done.


In [ ]:
%cd L2F_Challenge_YashGadhia/
!git checkout setpoints

/content/L2F_Challenge_YashGadhia/L2F_Challenge_YashGadhia/L2F_Challenge_YashGadhia
Branch 'setpoints' set up to track remote branch 'setpoints' from 'origin'.
Switched to a new branch 'setpoints'


In [ ]:
!pip install jsbsim
!python3 setup.py install
!pip install -e ./

jsbsim-1.1.11/aircraft/
jsbsim-1.1.11/aircraft/737/
jsbsim-1.1.11/aircraft/737/737.xml
jsbsim-1.1.11/aircraft/737/INSTALL
jsbsim-1.1.11/aircraft/737/cruise_init.xml
jsbsim-1.1.11/aircraft/737/cruise_steady_turn_init.xml
jsbsim-1.1.11/aircraft/737/reset00.xml
jsbsim-1.1.11/aircraft/A320/
jsbsim-1.1.11/aircraft/A320/A320.xml
jsbsim-1.1.11/aircraft/A320/INSTALL
jsbsim-1.1.11/aircraft/A4/
jsbsim-1.1.11/aircraft/A4/A4.xml
jsbsim-1.1.11/aircraft/A4/INSTALL
jsbsim-1.1.11/aircraft/B17/
jsbsim-1.1.11/aircraft/B17/B17.xml
jsbsim-1.1.11/aircraft/B17/INSTALL
jsbsim-1.1.11/aircraft/B17/reset00.xml
jsbsim-1.1.11/aircraft/B747/
jsbsim-1.1.11/aircraft/B747/B747.xml
jsbsim-1.1.11/aircraft/B747/INSTALL
jsbsim-1.1.11/aircraft/B747/reset00.xml
jsbsim-1.1.11/aircraft/Boeing314/
jsbsim-1.1.11/aircraft/Boeing314/Boeing314.xml
jsbsim-1.1.11/aircraft/Boeing314/INSTALL
jsbsim-1.1.11/aircraft/C130/
jsbsim-1.1.11/aircraft/C130/C130.xml
jsbsim-1.1.11/aircraft/C130/INSTALL
jsbsim-1.1.11/aircraft/C130/reset00.xml
js

In [ ]:
import gym
import gym_jsbsim
from gym_jsbsim.catalogs.catalog import Catalog as c
import numpy as np
import math
from stable_baselines3 import DQN, A2C, DDPG, HER, PPO, SAC, TD3
from stable_baselines3.common.env_checker import check_env

## Main code

Wrappers to make observations/actions conform to stable-baselines3 requirements

In [ ]:
class Obs_TupleToBoxWrapper(gym.ObservationWrapper):
    def __init__(self, env, test):
        super().__init__(env)
        self.test = test
        low = np.empty(shape=(0,))
        high = np.empty(shape=(0,))
        for i in env.observation_space:
          low = np.concatenate([low,i.low])
          high = np.concatenate([high,i.high])
        low = np.concatenate([low,-1.0])
        high = np.concatenate([high,1.0])
        self.observation_space = gym.spaces.Box(low=low, high=high, dtype="float")
        
    
    def observation(self, obs):
        new_obs = np.empty(shape=(0,))
        for i in obs:
          new_obs = np.concatenate([new_obs,i])
        new_obs = np.concatenate([new_obs,self.test(0.5)])
        return new_obs

In [ ]:
def test(i):
  return i

In [ ]:
class Act_TupleToBoxWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
        low = np.empty(shape=(0,))
        high = np.empty(shape=(0,))
        for i in env.action_space:
          low = np.concatenate([low,i.low])
          high = np.concatenate([high,i.high])
        self.action_space = gym.spaces.Box(low=low, high=high, dtype="float")
        
    
    def action(self, act):
#        print(act)
#        new_act = np.empty(shape=(0,))
#        for i in act:
#          new_act = np.concatenate([new_act,i])
        return act

Reward wrapper  
- Will use MSE(($dr^*$, $d\theta ^*$), ($dr_{obs}$, $d\theta_{obs}$))

Trajectory function
- tries to connect current heading to target heading via a circular path
- provides above path as incremental ($\Delta r$, $Δθ$) 
- input turning time, speed (fps), angle (deg), dt (which is simulation time corresponding to each iteration)

In [ ]:
env = Act_TupleToBoxWrapper(Obs_TupleToBoxWrapper(gym.make("GymJsbsim-HeadingControlTask-v0")))

Debug: 0
is_debug is True


TypeError: ignored

In [ ]:
check_env(env)

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log="./ppo_tensorboard/")
%load_ext tensorboard
%tensorboard --logdir ./ppo_tensorboard/

model.learn(total_timesteps=10000000)
model.save("/content/drive/Shareddrives/Learn_to_fly/ppo_model_setpoints_1")
del model

In [ ]:
model = PPO.load("/content/drive/Shareddrives/Learn_to_fly/ppo_model_setpoints_1", env=env)
episode_reward = 0
state = env.reset()
print("Initial State =", state)
done = False
while not done:
  action, _ = model.predict(state, deterministic=True)
  state, reward, done, _ = env.step(action)
  episode_reward += reward
  print("action =", action, " ---> State =", state, " : Reward =", reward)
print("Simulation Time = "+"{:.3f}".format(env.get_sim_time())+" sec")
print("Episode Reward = "+"{:.3f}".format(episode_reward))